# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df_cities = pd.read_csv('output_data/cities.csv')
df_cities.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Egvekinot,66.3167,-179.1667,-2.39,83,100,5.64,RU,1664213277
1,Muravlenko,63.7898,74.5230,-2.52,75,80,4.77,RU,1664213279
2,Mayumba,-3.4320,10.6554,24.63,81,35,3.12,GA,1664213281
3,Ocós,14.5094,-92.1933,30.10,79,100,3.31,GT,1664213282
4,Dicabisagan,17.0818,122.4157,25.46,90,82,1.08,PH,1664213003


In [3]:
df_cities.shape

(443, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
figure_layout = {'width': '1000px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(center=(0,0),zoom_level=2,layout=figure_layout)
layer = gmaps.heatmap_layer(df_cities[['Lat', 'Lng']], weights=df_cities['Humidity'])
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [6]:
df_cities.describe()

,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Date
count,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,4.430000e+02
mean,17.985774,24.757412,18.262235,71.040632,56.225734,3.795734,1.664214e+09
std,34.288365,91.227112,10.244114,21.574205,37.619235,2.758007,2.143333e+02
min,-54.800000,-179.166700,-8.620000,1.000000,0.000000,0.000000,1.664213e+09
25%,-11.323600,-56.162200,10.600000,61.000000,20.000000,2.000000,1.664213e+09
50%,18.800300,30.717800,21.020000,77.000000,63.000000,3.180000,1.664214e+09
75%,48.884150,110.266950,25.275000,87.000000,97.000000,5.020000,1.664214e+09
max,78.218600,178.416700,40.840000,100.000000,100.000000,21.440000,1.664214e+09


In [7]:
# & (df_cities.Max_Temp>70) &(df_cities.Wind_Speed<10) &(df_cities.Cloudiness<10)
df_candidate = df_cities.loc[(df_cities.Max_Temp<25) & (df_cities.Max_Temp>20) &(df_cities.Wind_Speed<10) &(df_cities.Cloudiness==0), :]
df_candidate.shape

(17, 9)

In [8]:
df_candidate

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
28,Inhambane,-23.8650,35.3833,23.80,86,0,7.77,MZ,1664213330
41,Ambovombe,-25.1667,46.0833,24.30,61,0,2.44,MG,1664213354
45,Birjand,32.8663,59.2211,21.26,23,0,5.14,IR,1664213361
116,Koungou,-12.7336,45.2042,23.84,78,0,0.00,YT,1664213482
133,Yachimata,35.6500,140.3167,22.55,94,0,1.54,JP,1664213511
188,Dāmghān,36.1683,54.3480,22.79,48,0,2.46,IR,1664213605
216,Mastic Beach,40.7668,-72.8521,23.60,68,0,7.72,US,1664213653
227,Antalya Province,36.7741,30.7178,24.49,67,0,0.00,TR,1664213459
233,Semnan,35.5729,53.3971,24.34,34,0,1.37,IR,1664213683
239,Beloha,-25.1667,45.0500,23.58,60,0,2.03,MG,1664213692


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
print('Start Retriving Hotel Data...')
print('-----------------------------')
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

lst_dicts = []
# Search for the first hotel in each coordinate
for _, r in df_candidate.iterrows():
    url = '{}?location={}%2C{}&radius=5000&type=hotel&key={}'.format(base_url, r['Lat'], r['Lng'], g_key)
    print('Retrieving hotel for: {}, {}'.format(r['City'], r['Country']))
    d = requests.request("GET", url).json()
    
    if not d or d['status'] != 'OK':
        print('Error retrieving hotel for {}!'.format(r['City']))
        continue
    hotel = d['results'][0]
    # print(res.json())
    lst_dicts.append({
        'Hotel Name': hotel['name'],
        'Lat': hotel['geometry']['location']['lat'],
        'Lng': hotel['geometry']['location']['lng'],
        'City': r['City'],
        'Country': r['Country']
    })
print('-----------------------------')
print('Retriving Hotel Data Done')
print('{} Hotels info retried.'.format(len(lst_dicts)))

Start Retriving Hotel Data...
-----------------------------
Retrieving hotel for: Inhambane, MZ
Retrieving hotel for: Ambovombe, MG
Retrieving hotel for: Birjand, IR
Retrieving hotel for: Koungou, YT
Retrieving hotel for: Yachimata, JP
Retrieving hotel for: Dāmghān, IR
Retrieving hotel for: Mastic Beach, US
Retrieving hotel for: Antalya Province, TR
Retrieving hotel for: Semnan, IR
Retrieving hotel for: Beloha, MG
Retrieving hotel for: Neyshabur, IR
Retrieving hotel for: Tateyama, JP
Retrieving hotel for: Katsuura, JP
Retrieving hotel for: Casper, US
Retrieving hotel for: Gilgit, PK
Retrieving hotel for: Korla, CN
Retrieving hotel for: Cap Malheureux, MU
-----------------------------
Retriving Hotel Data Done
17 Hotels info retried.


In [10]:
hotel_df = pd.DataFrame(lst_dicts)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Maxixe,-23.860607,35.346803,Inhambane,MZ
1,Ambovombe,-25.172013,46.089712,Ambovombe,MG
2,Birjand,32.873317,59.216294,Birjand,IR
3,Mamoudzou,-12.780640,45.232696,Koungou,YT
4,Yachimata,35.660591,140.299173,Yachimata,JP
5,Damghan,36.162925,54.333980,Dāmghān,IR
6,Shirley,40.801488,-72.867603,Mastic Beach,US
7,Antalya Kemer Masaj,36.741298,30.721317,Antalya Province,TR
8,Semnan,35.576704,53.394917,Semnan,IR
9,Beloha,-25.169449,45.060656,Beloha,MG


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
figure_layout = {'width': '1000px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(center=(0,0),zoom_level=2,layout=figure_layout)
layer = gmaps.heatmap_layer(df_cities[['Lat', 'Lng']], weights=df_cities['Humidity'])
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(layer)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='1000px'))